# Numerical approach to the ballistic problem

<img src="/img_cannonball.png" width=400 height=200/>

## Information

This problem is about a canon ball being shot in the air. The ball goes up and falls down, following a parabolic trajectory. We neglect all forces except gravity. The mathematical set up of the problem is:

$\begin{equation}
\left\{ 
  \begin{aligned}
    &m \frac{du}{dt}=0\\
    &m \frac{dw}{dt}=-mg
  \end{aligned}
\right.
\end{equation}$


Which gives:

$\begin{equation}
\left\{ 
  \begin{aligned}
    &u = u_{0}\\
    &w = -gt+w_{0}
  \end{aligned}
\right.
\end{equation}$


We will solve this system numerically, by integrating forward in time. However, note the equations for $x$ and $z$ do have analytic solutions. They can be found by integration and substitution. The solutions are: \

$\begin{equation}
\left\{ 
  \begin{aligned}
    &x = u_{0}t \\
    &z = \frac{-g}{2u_{0}^{2}} x^{2} + \frac{w_{0}}{u_{0}} x
  \end{aligned}
\right.
\end{equation}$

In this notebook, we use the known solution for $x$. We integrate $w$ numerically to determine the $z$ trajectory. The numerical trajectory can be compared to the analytic solution.

In [ ]:
import os
import sys
import math
import time
import numpy as np
import matplotlib.pyplot as plt

Set up the parameters and initial values.

In [ ]:
xx0   = 0
zz0   = 0
uu0   = 5 #[m/s]
ww0   = 30 #[m/s]
grv   = 9.81 #[m/s2]

Set up the time stepping.

In [ ]:
tmend   = 10
deltat  = 0.1
nsteps  = int(tmend/deltat)

We can define the function describing the analytic trajectory solution.

In [ ]:
def analytic_ballistic(xpos,uinit,winit,gravity=9.81):
    '''
    Analytic parabola trajectory for the ballistic problem
    '''
    zpos = -(gravity/2)*(1/uinit**2)*xpos**2+(winit/uinit)*xpos
    return(zpos)

Now, we initialize our arrays for the analytic and the numerical solution.

In [ ]:
nmr_zzfull     = np.zeros(nsteps)
nmr_zzfull[0]  = zz0
atic_zzfull    = np.zeros(nsteps)
atic_zzfull[0] = zz0
xxfull         = np.zeros(nsteps)
xxfull[0]      = xx0

Step forward in time:

In [ ]:
### Forward Euler ###
for kk in range(nsteps-1):
    xxfull[kk+1]      = uu0*(kk+1)*deltat
    nmr_zzfull[kk+1]  = nmr_zzfull[kk]+deltat*(ww0-grv*(kk+1)*deltat)
    atic_zzfull[kk+1] = analytic_ballistic(xxfull[kk+1],uu0,ww0,grv)
    atic_zzfull[kk+1] = np.maximum(atic_zzfull[kk+1],0)
    nmr_zzfull[kk+1]  = np.maximum(nmr_zzfull[kk+1],0)

Check the error of the numerical scheme:

In [ ]:
resids_nmr = np.round(nmr_zzfull-atic_zzfull,3)
print(f'Maximum error: {resids_nmr[np.argmax(abs(resids_nmr))]} m')


Make a figure.

In [ ]:
fig = plt.figure(figsize=(6,5))
ax = plt.subplot(111)
ax.plot(xxfull,atic_zzfull,c='k',label='analytic trajectory')
ax.scatter(xxfull,nmr_zzfull,c='forestgreen',marker='x',s=10,label='numerical trajectory')
ax.legend(loc='best',fontsize=12)
ax.tick_params(which='major',axis='both',labelsize=11)
ax.set_xlabel('x [m]',fontsize=12)
ax.set_ylabel('z [m]',fontsize=12)
fig.tight_layout()